# 第三章总结 分类问题

## 1. 准备数据

* 准备数据
 * 数据集初探
 * 不用清洗，不用特征工程
 * 分离测试集

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

## 2. 选择模型、训练模型（二元分类器）

* 分类问题的适用模型：
 * SGD 随机梯度下降（Stochastic Gredient Dscending)
 * Decistion Treee 决定树

## 3. 对模型进行评估（二元分类器）

* 准确度 accuracy（不是很有效，尤其对有偏斜的数据）
* 混淆矩阵 confusion matrix （A实例被识别为B的次数）
* 精度（正类预测的准确率）
* 召回率、灵敏度、真正类率（检测到的正类比率）
* F1分数（精度与召回率的谐波平均），给予较小的值更大的权重
* 在召回率和精度之间平衡，以选择阀值：
 * 精度（召回率）-阀值曲线
 * 精度-召回率曲线（PR曲线）
* 比较多个模型的表现：
 * ROC曲线（受试者工作特征曲线、真正类率V.S.假正类率（1-特异度））
 * ROC AUC（ROC的曲线下面积）